<a href="https://colab.research.google.com/github/jeong2624/Deep-Learning-for-the-Life-Sciences-coding-transcription/blob/main/%08Chapter3_Machine_Learning_with_DeepChem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Machine Learnning with DeepChem**

해당 Chapter에서는 텐서플로 기반 머신러닝 라이브러리인 DeepChem으로 딥러닝을 생명과학 분야에 사용하는 방법을 소개.

DeepChem 라이브러리에는 생명과학에 특화된 모델, 알고리즘, 데이터셋이 포함되어 있다.

해당 라이브러리의 장점은 텐서플로 기반으로 하기 때문에 기존 머신러닝 생태계와 잘 융합된다는 점이다. 즉, DeepChem으로 작성한 코드도 텐서플로에서 작동할 수 있음!


##### <font color=red>📌 **케라스, 텐서플로, Pytorch 라이브러리를 사용하지 않는 이유?**</font>
- 라이브러리 개발자들은 핵심 사용자들이 원하는 기능을 제공하는 데 초점을 맞춤.
- 예를 들어 케라스 (Keras) 라이브러리는 이미지 처리, 텍스트 처리, 음성 분석에 대해 광범위한 지원을 제공하지만, 분자 데이터의 처리, 유전학 데이터 또는 현미경 이미지에 대한 지원은 미비함.

In [1]:
# 필요한 라이브러리 설치
!pip install --pre deepchem
!pip install numpy torch_geometric pytorch_lightning dm-haiku

### **1. DeepChem의 기본 데이터셋**

In [2]:
# 필요한 라이브러리 불러오기
import deepchem as dc
import numpy as np

* 데이터셋은 데이터 각각에 대한 정보를 포함하는 입력 벡터 x와 출력 벡터 y로 구성되어 있고 추가적인 기타 정보 등도 포함하고 있음.

* DeepChem은 데이터셋을 저장하기 위해 다양한 하위 클래스들을 사용하는데, NumpyDataset 클래스는 넘파이 배열로 편리하게 변환될 수 있어서 가장 많이 사용된다.

In [3]:
# 재현성을 위해 랜덤 시드 배정
# https://numpy.org/doc/stable/reference/random/generated/numpy.random.seed.html
np.random.seed(20231106)

# 간단한 넘파이 배열 만들기
x = np.random.random((4, 5))
y = np.random.random((4, 1))

In [4]:
# 입력 벡터 x 출력
x

array([[3.73654008e-01, 1.19435956e-04, 4.40377417e-01, 2.08107329e-01,
        4.57378254e-01],
       [2.17120676e-01, 5.92927536e-02, 5.47143218e-01, 8.38110010e-01,
        2.47595065e-02],
       [8.46797765e-01, 4.20485983e-01, 3.64240020e-01, 8.14602497e-01,
        7.76671479e-01],
       [7.58009684e-02, 7.50731334e-01, 8.20427474e-01, 1.28505681e-01,
        7.05276497e-01]])

In [5]:
# 출력 벡터 y 출력
y

array([[0.65499145],
       [0.42909778],
       [0.41402953],
       [0.74893433]])

In [6]:
# 해당 넘파이 배열을 NumpyDataset 객체로 저장
dataset = dc.data.NumpyDataset(x, y)

In [7]:
# NumpyDataset 객체 속에 저장된 원래의 넘파이 배열을 출력 (입력 벡터 x)
print(dataset.X)

[[3.73654008e-01 1.19435956e-04 4.40377417e-01 2.08107329e-01
  4.57378254e-01]
 [2.17120676e-01 5.92927536e-02 5.47143218e-01 8.38110010e-01
  2.47595065e-02]
 [8.46797765e-01 4.20485983e-01 3.64240020e-01 8.14602497e-01
  7.76671479e-01]
 [7.58009684e-02 7.50731334e-01 8.20427474e-01 1.28505681e-01
  7.05276497e-01]]


In [8]:
# NumpyDataset 객체 속에 저장된 원래의 넘파이 배열을 출력 (출력 벡터 y)
print(dataset.y)

[[0.65499145]
 [0.42909778]
 [0.41402953]
 [0.74893433]]


In [9]:
# 출력된 넘파이 배열이 원본과 동일한지 확인 (입력 벡터 x)
# np.array_eqaul 관련 정보 -> https://numpy.org/doc/stable/reference/generated/numpy.equal.html
# 비교 대상이 동일하면 True, 그렇지 않으면 False로 출력
np.array_equal(x, dataset.X)

True

In [10]:
# 출력된 넘파이 배열이 원본과 동일한지 확인 (출력 벡터 x)
# np.array_eqaul 관련 정보 -> https://numpy.org/doc/stable/reference/generated/numpy.equal.html
# 비교 대상이 동일하면 True, 그렇지 않으면 False로 출력
np.array_equal(x, dataset.X)

True

### **2. 독성 분자 예측 모델 만들기**

In [11]:
# Tox21 독성 데이터 불러오기
tox21_tasks, tox21_datasets, transformers = dc.molnet.load_tox21()

In [12]:
# tox21_tasks의 값 살펴보기
tox21_tasks

['NR-AR',
 'NR-AR-LBD',
 'NR-AhR',
 'NR-Aromatase',
 'NR-ER',
 'NR-ER-LBD',
 'NR-PPAR-gamma',
 'SR-ARE',
 'SR-ATAD5',
 'SR-HSE',
 'SR-MMP',
 'SR-p53']

In [13]:
# 해당 데이터에 생물학적 표적 (Target)이 총 몇 개가 있는지 확인
len(tox21_tasks)

12

In [14]:
# tox21_datasets 객체 살펴보기
# 해당 데이터에는 입력 및 출력 벡터의 shape, 생물학적 표적 (target) 정보, 각 표적에 대한 분자식 정보 등 들어 있다.
tox21_datasets

(<DiskDataset X.shape: (6264, 1024), y.shape: (6264, 12), w.shape: (6264, 12), task_names: ['NR-AR' 'NR-AR-LBD' 'NR-AhR' ... 'SR-HSE' 'SR-MMP' 'SR-p53']>,
 <DiskDataset X.shape: (783, 1024), y.shape: (783, 12), w.shape: (783, 12), ids: ['N#C[C@@H]1CC(F)(F)CN1C(=O)CNC1CC2CCC(C1)N2c1ncccn1'
  'CN(C)C(=O)NC1(c2ccccc2)CCN(CCC[C@@]2(c3ccc(Cl)c(Cl)c3)CCCN(C(=O)c3ccccc3)C2)CC1'
  'CSc1nnc(C(C)(C)C)c(=O)n1N' ...
  'O=C(O[C@H]1CN2CCC1CC2)N1CCc2ccccc2[C@@H]1c1ccccc1'
  'C#C[C@]1(O)CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@@H]4[C@H]3C(=C)C[C@@]21CC'
  'NC(=O)C(c1ccccc1)(c1ccccc1)[C@@H]1CCN(CCc2ccc3c(c2)CCO3)C1'], task_names: ['NR-AR' 'NR-AR-LBD' 'NR-AhR' ... 'SR-HSE' 'SR-MMP' 'SR-p53']>,
 <DiskDataset X.shape: (784, 1024), y.shape: (784, 12), w.shape: (784, 12), ids: ['CC1(C)S[C@@H]2[C@H](NC(=O)Cc3ccccc3)C(=O)N2[C@H]1C(=O)O.CC1(C)S[C@@H]2[C@H](NC(=O)Cc3ccccc3)C(=O)N2[C@H]1C(=O)O.c1ccc(CNCCNCc2ccccc2)cc1'
  'CC(C)(c1ccc(Oc2ccc3c(c2)C(=O)OC3=O)cc1)c1ccc(Oc2ccc3c(c2)C(=O)OC3=O)cc1'
  'Cc1cc(C(C)(C)C)c(O)c(C)c1

In [15]:
# 모델 학습을 위해 데이터셋을 분리
# tox21_datasets 객체에는 train, validation, test dataset이 들어 있음.
# 이것의 장점은 데이터셋을 반복해서 불러올 필요가 없음.
train_dataset, valid_dataset, test_dataset = tox21_datasets

In [16]:
# 각 데이터셋의 입력 벡터 shape 확인 (train_dataset)
train_dataset.X.shape

(6264, 1024)

In [17]:
# 각 데이터셋의 입력 벡터 shape 확인 (valid_dataset)
valid_dataset.X.shape

(783, 1024)

In [18]:
# 각 데이터셋의 입력 벡터 shape 확인 (test_dataset)
test_dataset.X.shape

(784, 1024)

In [19]:
# 각 데이터셋의 출력 벡터 shape 확인 (train_dataset)
np.shape(train_dataset.y)

(6264, 12)

In [20]:
# 각 데이터셋의 출력 벡터 shape 확인 (valid_dataset)
np.shape(valid_dataset.y)

(783, 12)

In [21]:
# 각 데이터셋의 출력 벡터 shape 확인 (test_dataset)
np.shape(test_dataset.y)

(784, 12)

In [22]:
# 해당 데이터셋에 결측치가 존재하는지 확인
# 총 11,521개의 element들이 결측치를 갖고 있음을 확인
np.count_nonzero(train_dataset.w == 0)

11521

In [23]:
# transformers 변수는 원본 데이터셋을 수정한 객체.
# DeepChem은 원본 데이터를 변환하는 유용한 도구를 제공
# 이 경우 어떤 도구가 사용됐는지 확인 -> BalancingTransformer : 가중치 행렬을 조정함으로서 불균형한 데이터셋을 보완하는 데 사용
transformers

In [24]:
# 다중 분류 모델 생성
# https://www.researchgate.net/figure/Example-code-for-benchmark-evaluation-with-DeepChem-multiple-methods-are-provided-for_fig1_314182452
model = dc.models.MultitaskClassifier(
    n_tasks = 12, # 12개의 데이터 포인트 (Label)
    n_features = 1024, # 각 샘플의 feature 개수
    layer_sizes = [1000], # 신경망의 hidden layer의 개수와 너비를 설정 (여기서는 너비가 1,000인 하나의 hidden layer 사용)
    dropouts = [.25], # 몇 개의 신경망 hidden layer 층을 임의로 삭제할 것인가?
    learning_rate = 0.001, # 한 번 학습할 때 얼마만큼 학습할 것인가?
    batch_size = 50 # 모델 학습 중 parameter 업데이트 할 때 사용할 데이터의 개수
)

In [25]:
# 모델 학습
# nb_epoch = 10 : 한 번에 epoch의 경사 하강법 학습을 10번 수행한다는 의미
# epoch : 데이터셋의 모든 샘플이 학습 알고리즘을 한 번 통과한다는 것을 의미
model.fit(train_dataset, nb_epoch = 10)

0.35574881235758465

In [26]:
# ROC AUC 점수를 사용해 모델의 성능을 평가
metric = dc.metrics.Metric(dc.metrics.roc_auc_score, np.mean)

In [27]:
# train, test 데이터셋에 대한 모델 성능 평가
train_scores = model.evaluate(train_dataset, [metric], transformers)
test_scores = model.evaluate(test_dataset, [metric], transformers)

In [28]:
# 모델의 ROC AUC 점수 출력
print(train_scores)
print(test_scores)

{'mean-roc_auc_score': 0.9704947629837704}
{'mean-roc_auc_score': 0.6721887166755774}


### **3. MNIST 데이터셋으로 필기 인식 모델 만들기**

* MNIST 필기 인식 데이터셋은 필기된 숫자를 올바르게 분류하는 머신러닝에 많이 사용됨.

* MNIST 데이터셋에는 0에서 9까지의 숫자에 해당되는 28 × 28 픽셀의 흑백 이미지가 학습 데이터에 60,000개, 테스트 데이터에 10,000개씩 들어 있음.

In [29]:
# 모델 분석에 필요한 라이브러리 불러오기
import tensorflow as tf
import tensorflow.keras.layers as layers

In [30]:
# MNIST 데이터 불러오기
# MNIST 객체에는 train, test dataset이 들어 있음.
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# 해당 데이터는 Categorical (범주형) 데이터로 만들어져 있기 때문에 모델이 학습할 수 있는 데이터 format으로 변환하기 위해 one-hot 인코딩 진행.
# https://wikidocs.net/22647
y_train = tf.one_hot(y_train, 10).numpy()
y_test = tf.one_hot(y_test, 10).numpy()

In [31]:
# 각 데이터셋을 train, test 객체로 변환하기
train_dataset = dc.data.NumpyDataset(x_train, y_train)
test_dataset = dc.data.NumpyDataset(x_test, y_test)

In [32]:
# Input size => 28 (가로) × 28 (세로) × 1 (흑백 이미지)
features = tf.keras.Input(shape = (28, 28, 1))

# 각 샘플에 2차원 합성곱 (2D convolution) 적용 / 활성 함수를 ReLU로 설정
conv2d_1 = layers.Conv2D(filters = 32, kernel_size = 5, activation = tf.nn.relu)(features) # 차원 : 28 -> 32 , kernel : 5 × 5
conv2d_2 = layers.Conv2D(filters = 64, kernel_size = 5, activation = tf.nn.relu)(conv2d_1) # 차원 : 32 -> 64, kernel : 5 × 5

# Conv2d 레이어의 출력은 2차원 벡터이므로 1차원 벡터로 변환하기 위해 Flatten() 함수 사용
flatten = layers.Flatten()(conv2d_2)
dense1 = layers.Dense(units = 1024, activation = tf.nn.relu)(flatten)
dense2 = layers.Dense(units = 10, activation = None)(dense1)

# Output size : 1 × 1 (Label)
# 차원 : 64 -> 1
output = layers.Activation(tf.math.softmax)(dense2)

# CNN 모델 생성
keras_model = tf.keras.Model(inputs = features, outputs = [output, dense2])
model = dc.models.KerasModel(
    keras_model,
    loss = dc.models.losses.SoftmaxCrossEntropy(), # 손실함수를 소프트맥스 교차 엔트로피로 설정
    output_types = ['prediction', 'loss'], # 모델 성능 평가
    model_dir = 'mnist') # model_dir : 모델의 매개변수를 저장하는 폴더를 지정

In [ ]:
# 모델 학습
model.fit(train_dataset, nb_epoch = 10)

In [ ]:
# Train, Test 데이터셋에 대한 모델 성능 평가 (Accuracy)
metric = dc.metrics.Metric(dc.metrics.accuracy_score)
train_scores = model.evaluate(train_dataset, [metric])
test_scores = model.evaluate(test_dataset, [metric])
print(train_scores)
print(test_scores)